In [1]:
import math
from datetime import timedelta, datetime
from operator import attrgetter

import matplotlib.pyplot as plt
import numpy as np
import trajan as ta
import xarray as xr
from IPython.display import HTML
from matplotlib.animation import FuncAnimation

import os
# from salishsea_tools import nc_tools, places

import parcels

In [2]:

path = {'NEMO': '/results2/SalishSea/nowcast-green.202111/',
'coords': '/ocean/vvalenzuela/MOAD/grid/coordinates_seagrid_SalishSea201702.nc',
'mask': '/ocean/vvalenzuela/MOAD/grid2/mesh_mask202108_TDV.nc',
# 'out': '/home/vvalenzuela/MOAD/Ocean_Parcels/results/Mixing_Advection_Sinking',
'bat': '/ocean/vvalenzuela/MOAD/grid/bathymetry_202108.nc'}
# 'home': '/home/vvalenzuela/MOAD/analysis-vicente/Ocean_Parcels',}

coords, mask = [xr.open_dataset(path[key], decode_times=False) for key in ('coords', 'mask')]
gridlon, gridlat = [coords[key][0, ...].values for key in ('glamt', 'gphit')]
tmask = mask.tmask[0, 0, ...].values

# lon, lat = places.PLACES[location]['lon lat']
# def filename_set(start,length,varlist=['U','V','W'],local=0):
#     '''filename,variables,dimensions = filename_set(start,duration,varlist=['U','V','W'],local=1)
#     Modify function to include more default variables
#     define start as: e.g, datetime(2018, 1, 17)
#     length= number of days'''
    
#     duration = timedelta(days=length)
#     #Build filenames
#     paths = path(local)
#     Rlist,Tlist,Ulist, Vlist, Wlist = [], [], [], [], []
#     # Waveslist = []
#     # Flist = []
#    #


# def make_prefix(date, path, res='h'):
#     """Construct path prefix for local SalishSeaCast results given date object and paths dict
#     e.g., /results2/SalishSea/nowcast-green.201905/daymonthyear/SalishSea_1h_yyyymmdd_yyyymmdd
#     """

#     datestr = '_'.join(np.repeat(date.strftime('%Y%m%d'), 2))
#     folder = date.strftime("%d%b%y").lower()
#     prefix = os.path.join(path, f'{folder}/SalishSea_1{res}_{datestr}')
    
#     return prefix

# paths = path
# start = datetime(2024, 5, 6)
# duration = timedelta(days=1)
# Ulist, Vlist, Wlist = [], [], []
# for day in range(duration.days):
#     path_NEMO = make_prefix(start + timedelta(days=day), paths['NEMO'])
#     # path_NEMO_d = make_prefix(start + timedelta(days=day), paths['NEMO'],res='d')
#     Ulist.append(path_NEMO + '_grid_U.nc')
#     Vlist.append(path_NEMO + '_grid_V.nc')
#     Wlist.append(path_NEMO + '_grid_W.nc')
#     # Waveslist.append(get_WW3_path(start + timedelta(days=day)))
#     # Flist.append(get_Fraser_path(start + timedelta(days=day)))


# # Load NEMO forcing 
# filenames = {
#     'U': {'lon': paths['coords'], 'lat': paths['coords'], 'depth': Wlist[0], 'data': Ulist},
#     'V': {'lon': paths['coords'], 'lat': paths['coords'], 'depth': Wlist[0], 'data': Vlist},
#     'W': {'lon': paths['coords'], 'lat': paths['coords'], 'depth': Wlist[0], 'data': Wlist},
#     'Bathy' : {'lon': paths['coords'], 'lat': paths['coords'], 'data': paths['bat']},
# }


In [3]:
def make_prefix(date, path, res='h'):
    """Construct path prefix for local SalishSeaCast results given date object and paths dict
    e.g., /results2/SalishSea/nowcast-green.201905/daymonthyear/SalishSea_1h_yyyymmdd_yyyymmdd
    """

    datestr = '_'.join(np.repeat(date.strftime('%Y%m%d'), 2))
    folder = date.strftime("%d%b%y").lower()
    prefix = os.path.join(path, f'{folder}/SalishSea_1{res}_{datestr}')
    
    return prefix

paths = path
# path_NEMO = make_prefix(start + timedelta(days=1), paths['NEMO'])

start = datetime(2024, 5, 6)
duration = timedelta(days=5)
Ulist, Vlist = [], []
# Wlist = path_NEMO + '_grid_W.nc'
# for day in range(duration.days):

path_NEMO = make_prefix(start + timedelta(days=1), paths['NEMO'])


In [4]:
filenames = {
    'U': {'lon': paths['coords'], 'lat': paths['coords'], 'depth': path_NEMO + '_grid_W.nc', 'data': path_NEMO + '_grid_U.nc'},
    'V': {'lon': paths['coords'], 'lat': paths['coords'], 'depth': path_NEMO + '_grid_W.nc', 'data': path_NEMO + '_grid_V.nc'},
    'W': {'lon': paths['coords'], 'lat': paths['coords'], 'depth': path_NEMO + '_grid_W.nc', 'data': path_NEMO + '_grid_W.nc'},
}

variables = {'U': 'vozocrtx', 'V': 'vomecrty','W': 'vovecrtz'}

dimensions = {"lat": "nav_lat", "lon": "nav_lon", "depth": "depthw", "time": "time_counter"}

# Finally, read in the fieldset using the FieldSet.from_netcdf function with the 
# above-defined filenames, variables and dimensions.

fieldset = parcels.FieldSet.from_netcdf(filenames, variables, dimensions)
print(fieldset)



/home/gwatts/miniforge3/envs/analysis-grace/lib/python3.13/site-packages/parcels/field.py:648: FileWarning: File /ocean/vvalenzuela/MOAD/grid/coordinates_seagrid_SalishSea201702.nc could not be decoded properly by xarray (version 2025.4.0). It will be opened with no decoding. Filling values might be wrongly parsed.
  with _grid_fb_class(
/home/gwatts/miniforge3/envs/analysis-grace/lib/python3.13/site-packages/parcels/field.py:648: FileWarning: File /ocean/vvalenzuela/MOAD/grid/coordinates_seagrid_SalishSea201702.nc could not be decoded properly by xarray (version 2025.4.0). It will be opened with no decoding. Filling values might be wrongly parsed.
  with _grid_fb_class(


<FieldSet>
    fields:
        <Field>
            name            : 'U'
            grid            : CurvilinearZGrid(lon=array([[-123.43, -123.42, -123.42, ..., -121.33, -121.32, -121.32],
               [-123.43, -123.43, -123.42, ..., -121.33, -121.33, -121.32],
               [-123.43, -123.43, -123.42, ..., -121.34, -121.33, -121.33],
               ...,
               [-126.39, -126.39, -126.38, ..., -124.34, -124.34, -124.34],
               [-126.40, -126.39, -126.39, ..., -124.35, -124.34, -124.34],
               [-126.40, -126.39, -126.39, ..., -124.35, -124.35, -124.34]], shape=(898, 398), dtype=float32), lat=array([[ 46.86,  46.86,  46.86, ...,  47.60,  47.60,  47.60],
               [ 46.86,  46.86,  46.87, ...,  47.60,  47.60,  47.60],
               [ 46.87,  46.87,  46.87, ...,  47.61,  47.61,  47.61],
               ...,
               [ 50.38,  50.38,  50.39, ...,  51.09,  51.10,  51.10],
               [ 50.39,  50.39,  50.39, ...,  51.10,  51.10,  51.10],
       

/home/gwatts/miniforge3/envs/analysis-grace/lib/python3.13/site-packages/parcels/field.py:648: FileWarning: File /ocean/vvalenzuela/MOAD/grid/coordinates_seagrid_SalishSea201702.nc could not be decoded properly by xarray (version 2025.4.0). It will be opened with no decoding. Filling values might be wrongly parsed.
  with _grid_fb_class(


In [5]:
pset = parcels.ParticleSet.from_line(
    fieldset=fieldset,
    pclass=parcels.JITParticle,
    size=5,  # releasing 5 particles
    depth=70,
    start=(-123.3722, 49.2022),  # releasing on a line: the start longitude and latitude
    finish=(-123.3100, 49.2060),  # releasing on a line: the end longitude and latitude
)
# pset = parcels.particleset(
#     fieldset=fieldset,
#     pclass=parcels.JITParticle,
#     lon=-123,
#     lat=49,
# )

In [6]:
# And finally execute the ParticleSet for 1 day using 4th order Runge-Kutta.

output_file = pset.ParticleFile(
    name="Parcels_SalishSeaCast_test.zarr", outputdt=timedelta(hours=1)
)
pset.execute(
    parcels.AdvectionRK4,
    runtime=duration,
    dt=timedelta(seconds=90),
    output_file=output_file,
)

AttributeError: module 'parcels' has no attribute 'AdvectionRK4_gw'

In [ ]:
# ds = xr.open_zarr("Parcels_SalishSeaCast_test.zarr")
ds.traj.plot(margin=2)
ax = plt.gca()
ax.set_xlim([-123.5*111430, -123*111430])
ax.set_ylim([48.7*128430, 49.2*128430])
plt.show()

In [ ]:
ds = xr.open_zarr("Parcels_SalishSeaCast_test.zarr")

plt.plot(ds.lon.T, ds.lat.T, ".-")
plt.xlabel("Zonal distance [m]")
plt.ylabel("Meridional distance [m]")
plt.show()